In [1]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation


# Training utils
from training_code.utils import utils_xgboost

# Optuna
import optuna
import joblib
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

c:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates=['FECHAHORA'])
validacion = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates=['FECHAHORA'])

# 6 horas

In [ ]:
horizon = [0.25]
resample = [5]
previous = [2]

stations = [1, 2, 3, 5, 6, 7, 8, 9, 10]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    #df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:

            for station_item in stations:

                estacion = station_item

                variables = ['MP1', 'MP2_5', 'MP10' 'AQI_MP10', 'AQI_MP2_5']

                dependent = ['AQI_MP2_5']

                number_of_features = len(variables)

                training_days = previous_item
                forecast_days = horizon_item

                if resample_item == 5:
                    samples_per_day = 288
                elif resample_item == 30:
                    samples_per_day = 48
                elif resample_item == 60:
                    samples_per_day = 24

                step = forecast_days*samples_per_day

                train_months = relativedelta(months = 12)

                input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
                output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
                train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

                X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                    estacion,
                                                                    train_months, 
                                                                    variables, 
                                                                    dependent, 
                                                                    train_test_samples, 
                                                                    input_samples, 
                                                                    output_samples, 
                                                                    number_of_features,
                                                                    step)


                def objective(trial):
        
                    params = {
                        'max_depth': trial.suggest_int('max_depth', 1, 9),
                        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
                        'n_estimators': trial.suggest_int('n_estimators', 10, 300),
                        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                        'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
                        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
                        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
                    }
                    
                    optuna_xgb_model = xgb.XGBRegressor(** params)
                    
                    trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

                    prediction = trained_xgb_model.predict(X_test)

                    MAPE = mean_absolute_percentage_error(prediction, y_test)
                    print('MAPE: ', MAPE)
                    MAE = np.mean(np.abs(prediction - y_test))
                    print('MAE: ', MAE)

                    MSE = mean_squared_error(prediction, y_test)
                    
                    return MSE

                study = optuna.create_study(direction='minimize')

                study.optimize(objective, n_trials = 100 )

                trial = study.best_trial

                joblib.dump(study, "optuna_studies/XGBOOST/experimento_full/estacion_"+str(station_item)+"_study_XGBOOST_mejoresparam40_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

                # VALIDATION

                params = study.best_params

                # study = joblib.load("optuna_studies/XGBOOST/experimento_full/study_XGBOOST_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

                xgb_model = xgb.XGBRegressor(** params)

                trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)
                
                prediction = trained_xgb_model.predict(X_test)
                
                predicciones = {}
                metricas = {}
                # guardamos los valores predecidos vs reales en un diccionario
                
                predicciones[station_item] = {'real' : y_test, 'prediccion': prediction}


                mean_real = y_test.mean()
                mean_prediction = prediction.mean()

                MAPE = mean_absolute_percentage_error(prediction, y_test) * 100
                MAE = mean_absolute_error(prediction, y_test)
                RMSE = mean_squared_error(prediction, y_test, squared = False)
                
                # guardamos las metricas en un diccionario
                
                metricas[station_item] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/XGBOOST/experimento_full/todaslasestaciones_menos_4_6hs_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []

            for i in stations:
                d = {'TARGET': predicciones[i]['real'].flatten(), 'FORECAST': predicciones[i]['prediccion'].flatten()}
                df_aux = pd.DataFrame(data = d)
                df_aux['ESTACION'] = i
                list_dfs.append(df_aux)

            df_predicciones = pd.concat(list_dfs)

            df_predicciones.info()

            df_predicciones.to_csv('datos/experimento_full/predicciones_todas_las_estaciones_menos_4_testing.csv')

In [ ]:
horizon = [1]
resample = [30]
previous = [2]

stations = [1, 2, 3, 5, 6, 7, 8, 9, 10]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    #df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:

            for station_item in stations:

                estacion = station_item

                variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5', 'TIPO', 'TRAFICO', 'HUMEDAD', 
                        'PRESION', 'TEMPERATURA','DIA_SEM']

                dependent = ['AQI_MP2_5']

                number_of_features = len(variables)

                training_days = previous_item
                forecast_days = horizon_item

                if resample_item == 5:
                    samples_per_day = 288
                elif resample_item == 30:
                    samples_per_day = 48
                elif resample_item == 60:
                    samples_per_day = 24

                step = forecast_days*samples_per_day

                train_months = relativedelta(months = 12)

                input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
                output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
                train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

                X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                    estacion,
                                                                    train_months, 
                                                                    variables, 
                                                                    dependent, 
                                                                    train_test_samples, 
                                                                    input_samples, 
                                                                    output_samples, 
                                                                    number_of_features,
                                                                    step)


                def objective(trial):
        
                    params = {
                        'max_depth': trial.suggest_int('max_depth', 1, 9),
                        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
                        'n_estimators': trial.suggest_int('n_estimators', 10, 300),
                        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                        'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
                        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
                        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
                    }
                    
                    optuna_xgb_model = xgb.XGBRegressor(** params)
                    
                    trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

                    prediction = trained_xgb_model.predict(X_test)

                    MAPE = mean_absolute_percentage_error(prediction, y_test)
                    print('MAPE: ', MAPE)
                    MAE = np.mean(np.abs(prediction - y_test))
                    print('MAE: ', MAE)

                    MSE = mean_squared_error(prediction, y_test)
                    
                    return MSE

                study = optuna.create_study(direction='minimize')

                study.optimize(objective, n_trials = 100 )

                trial = study.best_trial

                joblib.dump(study, "optuna_studies/XGBOOST/experimento_full/estacion_"+str(station_item)+"_study_XGBOOST_mejoresparam40_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

                # VALIDATION

                params = study.best_params

                # study = joblib.load("optuna_studies/XGBOOST/experimento_full/study_XGBOOST_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

                xgb_model = xgb.XGBRegressor(** params)

                trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)
                
                prediction = trained_xgb_model.predict(X_test)
                
                predicciones = {}
                metricas = {}
                # guardamos los valores predecidos vs reales en un diccionario
                
                predicciones[station_item] = {'real' : y_test, 'prediccion': prediction}


                mean_real = y_test.mean()
                mean_prediction = prediction.mean()

                MAPE = mean_absolute_percentage_error(prediction, y_test) * 100
                MAE = mean_absolute_error(prediction, y_test)
                RMSE = mean_squared_error(prediction, y_test, squared = False)
                
                # guardamos las metricas en un diccionario
                
                metricas[station_item] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

df_metricas = pd.DataFrame.from_dict(metricas)

df_metricas.to_csv('metrics/XGBOOST/experimento_full/todaslasestaciones_menos_4_6hs_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

list_dfs = []

for i in stations:
    d = {'TARGET': predicciones[i]['real'].flatten(), 'FORECAST': predicciones[i]['prediccion'].flatten()}
    df_aux = pd.DataFrame(data = d)
    df_aux['ESTACION'] = i
    list_dfs.append(df_aux)

df_predicciones = pd.concat(list_dfs)

df_predicciones.info()

df_predicciones.to_csv('datos/experimento_full/predicciones_todas_las_estaciones_menos_4_testing.csv')

# Validacion 6hs - 10 estaciones

In [18]:
lista = [datos,validacion]
df = pd.concat(lista)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1580842 entries, 0 to 264959
Data columns (total 28 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   ESTACION                1580842 non-null  int64         
 1   FECHAHORA               1580842 non-null  datetime64[ns]
 2   ANHO                    1580842 non-null  int64         
 3   DIA                     1580842 non-null  int64         
 4   MES                     1580842 non-null  int64         
 5   HORA                    1580842 non-null  int64         
 6   MINUTO                  1580842 non-null  int64         
 7   MP1                     1580842 non-null  float64       
 8   MP2_5                   1580842 non-null  float64       
 9   MP10                    1580842 non-null  float64       
 10  TEMPERATURA             1580842 non-null  float64       
 11  HUMEDAD                 1580842 non-null  float64       
 12  PRESION        

In [15]:
horizon_item = 0.25
resample_item = 5
previous_item = 2

stations = [1, 2, 3, 5, 6, 7, 8, 9, 10]

predicciones = {}
metricas = {}

for i in stations:

    estacion = i

    variables = ['MP1', 'MP2_5','AQI_MP10', 'AQI_MP2_5']

    dependent = ['AQI_MP2_5']

    number_of_features = len(variables)

    training_days = 2 
    forecast_days = 1/4
    samples_per_day = 288
    step = 288/4

    # Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
    train_months = relativedelta(months = 15)

    input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
    output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
    train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test



    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df, 
                                                                    estacion,
                                                                    train_months, 
                                                                    variables, 
                                                                    dependent, 
                                                                    train_test_samples, 
                                                                    input_samples, 
                                                                    output_samples, 
                                                                    number_of_features,
                                                                    step)


    study = joblib.load("optuna_studies/XGBOOST/experimento_full/Validaciones/estacion_"+str(i)+"_study_XGBOOST_mejoresparam40_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

    params = study.best_params

    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_test)

    # guardamos los valores predecidos vs reales en un diccionario

    predicciones[i] = {'real' : y_test, 'prediccion': prediction}

    mean_real = y_test.mean()
    mean_prediction = prediction.mean()

    MAPE = mean_absolute_percentage_error(prediction, y_test) * 100
    MAE = mean_absolute_error(prediction, y_test)
    RMSE = mean_squared_error(prediction, y_test, squared = False)

    # guardamos las metricas en un diccionario

    metricas[i] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

    print('ESTACION '+str(i)+':')
    print('prediction shape: ', prediction.shape)
    print('test shape: ', y_test.shape)
    print('MAE :', MAE)
    print('MAPE: ', MAPE)
    print('RMSE: ', RMSE)
    print('\n')
    print('media real: ', mean_real)
    print('media predecida: ', mean_prediction)
    print('\n')

df_metricas = pd.DataFrame.from_dict(metricas)

df_metricas.to_csv('metrics/XGBOOST/experimento_full/validation_xgboost_todaslasestaciones_5_0.25_2.csv')

list_dfs = []

for i in stations:
    d = {'TARGET': predicciones[i]['real'].flatten(), 'FORECAST': predicciones[i]['prediccion'].flatten()}
    df_aux = pd.DataFrame(data = d)
    df_aux['ESTACION'] = i
    list_dfs.append(df_aux)

df_predicciones = pd.concat(list_dfs)


df_predicciones.to_csv('datos/experimento_full/predicciones_todaslasestaciones_validation_xgboost_5_0.25_2.csv')



ESTACION 1:
prediction shape:  (360, 72)
test shape:  (360, 72)
MAE : 3.202421971033385
MAPE:  10.52100872488529
RMSE:  4.8534711228923


media real:  36.895833333333336
media predecida:  36.41405


ESTACION 2:
prediction shape:  (360, 72)
test shape:  (360, 72)
MAE : 2.4197551049568036
MAPE:  8.144040920238563
RMSE:  3.585456067274404


media real:  33.37260802469136
media predecida:  32.56632


ESTACION 3:
prediction shape:  (360, 72)
test shape:  (360, 72)
MAE : 2.7561510776296076
MAPE:  6.4389540985370015
RMSE:  4.499839014437153


media real:  45.66574074074074
media predecida:  45.85107


ESTACION 5:
prediction shape:  (360, 72)
test shape:  (360, 72)
MAE : 2.2862015185532747
MAPE:  7.812749814948501
RMSE:  3.8660099478084757


media real:  31.298881172839508
media predecida:  30.98463


ESTACION 6:
prediction shape:  (360, 72)
test shape:  (360, 72)
MAE : 4.295128290557567
MAPE:  8.120308467725543
RMSE:  7.871595885138699


media real:  54.965123456790124
media predecida:  53.46

In [21]:
lista = [datos,validacion]
df = pd.concat(lista)


horizon_item = 1
resample_item = 5
previous_item = 2

stations = [1, 2, 3, 5, 6, 7, 8, 9, 10]

predicciones = {}
metricas = {}

df_train = df.copy()

df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
            'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
            'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

df_train = df_train.set_index('FECHAHORA', drop = True)

lista_resample = []

if resample_item != 5: 

    r_i = str(resample_item) + 'T'

    for station in range(1,11):
        df_aux = df_train[df_train['ESTACION'] == station]

        df_aux = df_aux.resample(r_i).mean()
        lista_resample.append(df_aux)

    df_train = pd.concat(lista_resample)

df_train['ANHO'] = df_train.index.year
df_train['MES'] = df_train.index.month
df_train['HORA'] = df_train.index.hour
df_train['MINUTO'] = df_train.index.minute
df_train['DIA'] = df_train.index.day
df_train['FECHAHORA'] = df_train.index
df = df_train.reset_index(drop = True)

for i in stations:

    estacion = i

    variables = ['MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5']

    dependent = ['AQI_MP2_5']

    number_of_features = len(variables)

    training_days = 2 
    forecast_days = 1
    samples_per_day = 288
    step = 288

    # Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
    train_months = relativedelta(months = 15)

    input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
    output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
    train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test



    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df, 
                                                                    estacion,
                                                                    train_months, 
                                                                    variables, 
                                                                    dependent, 
                                                                    train_test_samples, 
                                                                    input_samples, 
                                                                    output_samples, 
                                                                    number_of_features,
                                                                    step)


    study = joblib.load("optuna_studies/XGBOOST/experimento_full/Validaciones/estacion_"+str(i)+"_study_XGBOOST_mejoresparam40_resample_30_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

    params = study.best_params

    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_test)

    # guardamos los valores predecidos vs reales en un diccionario

    predicciones[i] = {'real' : y_test, 'prediccion': prediction}

    mean_real = y_test.mean()
    mean_prediction = prediction.mean()

    MAPE = mean_absolute_percentage_error(prediction, y_test) * 100
    MAE = mean_absolute_error(prediction, y_test)
    RMSE = mean_squared_error(prediction, y_test, squared = False)

    # guardamos las metricas en un diccionario

    metricas[i] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

    print('ESTACION '+str(i)+':')
    print('prediction shape: ', prediction.shape)
    print('test shape: ', y_test.shape)
    print('MAE :', MAE)
    print('MAPE: ', MAPE)
    print('RMSE: ', RMSE)
    print('\n')
    print('media real: ', mean_real)
    print('media predecida: ', mean_prediction)
    print('\n')

df_metricas = pd.DataFrame.from_dict(metricas)

df_metricas.to_csv('metrics/XGBOOST/experimento_full/prueba_validation_xgboost_todaslasestaciones_5_1_2.csv')

list_dfs = []

for i in stations:
    d = {'TARGET': predicciones[i]['real'].flatten(), 'FORECAST': predicciones[i]['prediccion'].flatten()}
    df_aux = pd.DataFrame(data = d)
    df_aux['ESTACION'] = i
    list_dfs.append(df_aux)

df_predicciones = pd.concat(list_dfs)


df_predicciones.to_csv('datos/experimento_full/prueba_predicciones_todaslasestaciones_validation_xgboost_5_1_2.csv')



ESTACION 1:
prediction shape:  (90, 288)
test shape:  (90, 288)
MAE : 10.404782100702509
MAPE:  32.87986538457702
RMSE:  13.39423816298831


media real:  36.895833333333336
media predecida:  33.651154


ESTACION 2:
prediction shape:  (90, 288)
test shape:  (90, 288)
MAE : 6.2369663865110025
MAPE:  21.481068678294278
RMSE:  8.72257603632884


media real:  33.37260802469136
media predecida:  30.801903


ESTACION 3:
prediction shape:  (90, 288)
test shape:  (90, 288)
MAE : 7.124791970370729
MAPE:  16.50108850853668
RMSE:  9.406605864768617


media real:  45.66574074074074
media predecida:  45.90638


ESTACION 5:
prediction shape:  (90, 288)
test shape:  (90, 288)
MAE : 5.935631191914464
MAPE:  20.02892636679644
RMSE:  8.613564204261039


media real:  31.298881172839508
media predecida:  30.577255


ESTACION 6:
prediction shape:  (90, 288)
test shape:  (90, 288)
MAE : 11.380868103327574
MAPE:  21.449823748716565
RMSE:  18.030582388297915


media real:  54.965123456790124
media predecida:  